In [1]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import json
import dataset
import credentials
import sqlite3
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import pickle
from Naive_bayes_model import *

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
f = open('nb.pickle', 'rb')
NBayesClassifier = pickle.load(f)
f.close()

In [3]:
conn = sqlite3.connect('tweet2.db')
c = conn.cursor()

In [4]:
def pred(data):
    
    tweetProcessor = PreProcessTweets()
    preprocessedTrainingSet = tweetProcessor.processTweets(data)
    
    word_features = buildVocabulary(preprocessedTrainingSet)
    trainingFeatures = nltk.classify.apply_features(extract_features, preprocessedTrainingSet)
    
    print(preprocessedTrainingSet[0][0])
    
    NB = NBayesClassifier.classify(extract_features(preprocessedTrainingSet[0][0],word_features))
    
    return NB
    

In [5]:
class TwitterStreamer():
    
    def stream_tweets(self, fetched_tweets_filename, hash_tag_list):
        listener = StdOutListener(fetched_tweets_filename)
        auth = OAuthHandler(credentials.CONSUMER_KEY, credentials.CONSUMER_SECRET)
        auth.set_access_token(credentials.ACCESS_TOKEN, credentials.ACCESS_TOKEN_SECRET)
        
        stream = Stream(auth, listener)
        
        stream.filter(track=['accident'])
        
 
class StdOutListener(StreamListener):
    
    def on_status(self, status):
        print(status.text)

    def on_error(self, status):
        print(status)

    def on_data(self, data):
        all_data             = json.loads(data)
        created_at           = all_data['created_at']
        favorite_count       = all_data['favorite_count']
        favorited            = all_data['favorited']
        filter_level         = all_data['filter_level']
        lang                 = all_data['lang']
        retweet_count        = all_data['retweet_count']
        retweeted            = all_data['retweeted']
        source               = all_data['source']
        text                 = all_data['text']
        truncated            = all_data['truncated']
        user_created_at      = all_data['user']['created_at']
        user_followers_count = all_data['user']['followers_count']
        user_location        = all_data['user']['location']
        user_lang            = all_data['user']['lang']
        user_name            = all_data['user']['name']
        user_screen_name     = all_data['user']['screen_name']
        user_time_zone       = all_data['user']['time_zone']
        user_utc_offset      = all_data['user']['utc_offset']
        user_friends_count   = all_data['user']['friends_count']
        geo_enabled          = all_data['user']['geo_enabled']  
        coordinates          = all_data['coordinates']
        geo                  = all_data['geo']
        place                = all_data['place']
        if place is not None:
            place_name       = all_data['place']['name']
        else:
            place_name       = all_data['place']
        
        print(text)
        
        data = [{'text': text,'Classfication': ''}]
        
        info = pred(data)
        
        print(info)
                         
#         c.execute('''INSERT INTO tweets 
#         (created_at, favorite_count, favorited, filter_level, lang, 
#                          retweet_count, retweeted, source, text, truncated, user_created_at,  
#                          user_followers_count, user_location, user_lang, user_name, 
#                          user_screen_name, user_time_zone, user_friends_count,geo_enabled,coordinates,geo,place_name ) 
#             VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''', 
#             (created_at, favorite_count, favorited, filter_level, lang, retweet_count, 
#                          retweeted, source, text, truncated, user_created_at, 
#                          user_followers_count, user_location, user_lang, user_name, 
#                          user_screen_name, user_time_zone, user_friends_count,geo_enabled,coordinates,geo,place_name ))
        
        c.execute('''INSERT INTO tweet_class 
        (text, class) 
            VALUES (?,?)''', 
            (text, info))
        
        conn.commit()

In [6]:
if __name__ == '__main__':
    
    hash_tag_list = ['accident']
    fetched_tweets_filename = "tweets.json"
    
    twitter_streamer = TwitterStreamer()
    twitter_streamer.stream_tweets(fetched_tweets_filename, hash_tag_list)

RT @ArchivesNZ: #OnThisDay 1904 Gunner John Amos Palmer was killed and 5 others injured in an accident during firing practice at Fort Balla…
['rt', 'onthisday', '1904', 'gunner', 'john', 'amos', 'palmer', 'killed', '5', 'others', 'injured', 'accident', 'firing', 'practice', 'fort', 'balla…']
Can't Decide
@Jannelle12 The should’ve put cast in his head instead of his leg when he had the career ending motorcycle accident… https://t.co/2YGqGEy7AU
['’', 'put', 'cast', 'head', 'instead', 'leg', 'career', 'ending', 'motorcycle', 'accident…']
Can't Decide
EMS responding for an accident on southbound 61/67 at Autumn Oaks Drive... @kmoxnews #stltraffic
['ems', 'responding', 'accident', 'southbound', '61/67', 'autumn', 'oaks', 'drive', '...', 'stltraffic']
Can't Decide
RT @bavabinks: Mon grand frère à 17 ans il a volé la voiture de ma grand mere il l’a accidenté, il est rentrer il a reposer les clés d’la v…
['rt', 'mon', 'grand', 'frère', 'à', '17', 'ans', 'il', 'volé', 'la', 'voiture', 'de', 'gr

KeyboardInterrupt: 